# Intelligent Systems 2020: 5th  practical assignment 
## Logical Agents

Your name: Mark Melnic

Your VUnetID: mmc560

If you do not provide your name and VUnetID we will not accept your submission. 

### Learning objectives
At the end of this exercise you should be able to work with some basic fuzzy concepts, and implement and evaluate a simple probabilistic approach to playing Schnapsen. 

### Preliminaries

In this worksheet we will build a Fuzzy Logic knowledge-base and implement a simple probabilistic strategy for an agent to play Schnapsen. 


### Practicalities

Follow this Notebook step-by-step. 

Of course, you can do the exercises in any Programming Editor of your liking. But you do not have to. Feel free to simply write code in the Notebook. Please use your studentID+Assignment5.ipynb as the name of the Notebook.  

Note: unlike the courses dedicated to programming we will not evaluate the style of the programs. But we will, however, test your programs on other data that we provide, and your program should give the correct output to the test-data as well.

As was mentioned, the assignment is graded as pass/fail. To pass you need to have either a full working code or an explanation of what you tried and what didn't work for the tasks that you were unable to complete (you can use multi-line comments or a text cell).


## Calculating Fuzzyvalues for a Formula (KnowledgeBase)

The first task is to implement part of the algorithm to calculate a fuzzy value for a complex formula:  

> f(¬A) = 1 - f(A) <br>
> f(A v B) = max(f(A);f(B)) <br>
> f(A & B) = min(f(A);f(B)) <br>
> f(A -> B) = min(1;1 – f(A) + f(B)) <br>

We will do this in a very simplistic way, which should still give you an additional idea on how this can be done. The main restriction is that we will only calculate the value of formulas in Clause Normal Form. This means we do not have to implement the rule for implication, and we do have a very simple structure of formulas: 

> (L11 v L12 v L13 v .... v L1n) <br>
> & (L21 v L22 v L23 v .... v L2n) <br>
> & ... 

where each of the L's is a Literal, or in other words an Atom, or a negation of an Atom. So, all that needs to be done is to be able to: <br>

1) assign a fuzzy value V to an atom, or (1 - V) value in case it is a negation. <br> 
2) calculate the fuzzy value of a disjunction of values for Literals, and  <br>
3) calculate the fuzzy value of a conjuction of values for disjuctions. <br>

There are two steps that need to be taken and they will have to be done at two different places in the fuzzykb.py file. Open this file in a Python editor of your choice. First, we will have to define the value of a negated atom. 

#### Task 1A: Fuzzy negation
Go to the definition of the NegFuzzySymbol class. Here in line 52, given a fuzzy value "value", the value of a negation has to be defined. Replace the line "???" with the correct code, and add the snippet to the following cell:

return 1 - value

#### Task 1B: Conjunction and Disjunction 
The task is now to: 
1) define the fuzzy value of Clause based on the values of the symbols (you get those by calling symbol.value() for all symbols in a clause)<br>
2) define the fuzzy value of the Knowlegde base (i.e. the conjunction of all the clauses) 

Go to the definition of the fuzzyvalue() function. Here in line 120 replace the lines "???" with the correct code, and add the snippet you wrote to the following cell:

minvalue = []
for clause in self._clauses:
    clausevalue = []
    for symbol in clause:
        clausevalue.append(symbol.value())
    minvalue.append(min(clausevalue))
return max(minvalue)

Once you have succesfully finished the implementations of these methods, you should be able to run the following code.

In [1]:
import sys
from bots.kbbot.fuzzykb import fuzzyKB, FuzzySymbol

# Define our symbols
A = FuzzySymbol('A',0.3)
B = FuzzySymbol('B',0.8)
C = FuzzySymbol('C',0.6)

# Create a new knowledge base
kb = fuzzyKB()

# Add clauses
kb.add_clause(A, B, C)
kb.add_clause(~A, B)
kb.add_clause(~B, C)
kb.add_clause(B, ~C)

# Calculate the fuzzy value of the knowledge base (the conjunction of all the clauses).
print("f(kb):", kb.fuzzyvalue())

f(kb): 0.6


### Task 2

The next task is to model the properties of Schnapsen cards in a simple Fuzzy Representation, with the following propositions: 

> HC = HighCard<br>
> VHC = VeryHighCard<br>
> Avg = AverageCard<br>
> Exp = ExpensiveCard<br>
> Chp = CheapCard 

Now let us build a fuzzy knowledge-base with values for a Queen (there is not one correct answer, but try to model this as close as possible to the true value and cost of a Queen in Schnapsen. 

a) First, let us reset the knowledge base, and define our fuzzy variables for a Queen: 

In [2]:
kb = fuzzyKB()
# Define our symbols
HC = FuzzySymbol('Q',0.3)
VHC = FuzzySymbol('Q',0.1)
Avg = FuzzySymbol('Q',0.5)
Low = FuzzySymbol('Q',0.7)
Chp = FuzzySymbol('Q',0.9)

b) Let us define now a set of cards, and check to what degree our queen is a member of this set. We define the set of cards as a set that are   
> 1. are either low, high or very high<br>
> 2. neither very high, nore average<br>
> 3. either not low or cheap, AND  <br>
> 4. neither cheap, nor high

In [3]:
#Add clauses
kb.add_clause(Low, HC, VHC)
kb.add_clause(~VHC, ~Avg)
kb.add_clause(~Low, Chp)
kb.add_clause(~Chp, ~HC)

Now run the script and report the output in the cell after the code.

In [4]:
# Calculate the fuzzy value of the knowledge base (the conjunction of all the clauses).
print("f(kb):", kb.fuzzyvalue())

f(kb): 0.7


Explain in your own words what does this output mean:

This output means that the Queen is a member of this set with a probability of 0.7 (70%).

## A Probability-based Bot. 

Now it is time to move to a real rational agent that plays Schnapsen, and to implement a probabilistic bot. The idea of this bot is described on the Canvas page for this assignment and in the probability.py file. 

### Tasks 3:
You will have to finish the implementation of a bot that uses probabilistic reasoning to determine its next move. All you have to do is fill in the missing code at the lines marked with "???". At these spots, we explain what you will have to do, but we strongly recommend that you also have a careful look at the entire bot, and the documentation of the code to get the overall idea.  

But first, let us not forget to initialise it:

In [5]:
import sys, random

from api import State, engine, util

Now you need to start coding. In the directory /bots, you will have to make a subdirectory called probability, and copy the provided file probability.py into this new directory. 

The next step is to finish the implemenation of this file. Open it in your favourite Python Editor and fill in the gaps. 

First, in line 101, you will need to implement the probability that the opponent has a problemCard. 

In line 107, you will have to update the maximal probability value and the chosen move accordingly. 

Now we can run a game between rand and your new bot, to check whether everything works fine. 

In [6]:
# Choose your first player
player1 = "rand"
player2 = "probability"
# Decide in which phase you want to start the game. 
startphase = 1
# Decide whether you want verbose output or not 
verbose=True 

#And here you run a game on the engine. 
engine.play(util.load_player(player1),util.load_player(player2), state=State.generate(phase=startphase), max_time=10000, verbose=verbose)

player1: <bots.rand.rand.Bot object at 0x0000015BB94123A0>
player2: <bots.probability.probability.Bot object at 0x0000015BB942E1C0>
*   Player 2 plays: AC
The game is in phase: 1
Player 1's points: 0, pending: 0
Player 2's points: 0, pending: 0
The trump suit is: C
Player 1's hand: KC AD JD KH KS
Player 2's hand: AC 10C AH 10H AS
There are 10 cards in the stock
Player 2 has played card: A of C

*   Player 1 plays: AD
The game is in phase: 1
Player 1's points: 0, pending: 0
Player 2's points: 22, pending: 0
The trump suit is: C
Player 1's hand: KC JD KH 10S KS
Player 2's hand: 10C AH 10H JH AS
There are 8 cards in the stock

*   Player 2 plays: 10C
The game is in phase: 1
Player 1's points: 0, pending: 0
Player 2's points: 22, pending: 0
The trump suit is: C
Player 1's hand: KC JD KH 10S KS
Player 2's hand: 10C AH 10H JH AS
There are 8 cards in the stock
Player 2 has played card: 10 of C

*   Player 1 plays: KC
The game is in phase: 1
Player 1's points: 0, pending: 0
Player 2's points: 

(2, 3)

Provide in the following cell the code that you have written to make the probabilistic bot work:

```
pc = len(problematicCards)
probability = ((u-pc)/u)*((u-1-pc)/(u-1))*((u-2-pc)/(u-2))*((u-3-pc)/(u-3))*((u-4-pc)/(u-4))
if (move[0] is not None and probability > maxProbability):
    maxProbability = probability
    chosen_move = move
```

An alternative might be to run games from the command-line in your terminal. To run a game between rand and probability just run: 

> python play.py -1 rand -2 probability 

or check out line python play.py -h for more details. Alternatively you can run a tournament with 
 
> python tournament.py -p rand,probability <br>
(and again check for help with: python tournament.py -h). 

Of course, you can also run the tournament in this notebook with the code that we provided in the last assignment if you prefer it that way. 

### Task 4

Run a tournament against some of the other bots, e.g. rand, kbbot or rdeep. Describe your findings in the next cell. 

Most of the times, in tournaments, the bot outperforms rand with a high margin, though not as well against rdeep. Kbbot is tricky in this situations, because the bots have a 50/50 win rate, which means they are almost on the same level.

## Utility

Unless we are very much mistaken, your new probability bot will not perform very well. One reason for this is that our probabilistic strategy has a serious flaw: the Aces and 10s have a high probability of not having a higher card of the same suit, so that our strategy will pick valuable cards in phase 1. This is a high-gain, but also a high-risk strategy, as a reasonably good opponent would trump those cards and win valuable points. 

One possible solution is to combine the probability of a card being easily beaten with the costs it takes to loose such a card. We do this by introducing a notion of utily, which simply devides the probability of being good by the costs of a potential loss of the played card. 

### Task 5 

Now you need to do a bit of coding again. In the directory /bots, you will have to make a subdirectory called utility, and copy the provided utility.py file into this new directory.

The next step is to finish the implemenation of this file. Open it in your favourite Python Editor and fill in the gaps. 

First, you will need to copy the solutions from probability.py to utility.py in lines 97 and 115, but now also a single line in 108. 

Provide the written lines of code in the following cell. 

```
utility = abs(u-pc)/u*abs(u-1-pc)/abs(u-1)*abs(u-2-pc)/abs(u-2)*abs(u-3-pc)/abs(u-3)*abs(u-4-pc)/abs(u-4)

score = [11, 10, 4, 3, 2]
if (move[0] is not None):
    rank = move[0] % 5
    points = score[rank]
    utility /= points
else:
    utility = 0.0

if (move[0] is not None and utility > maxUtility):
    maxUtility = utility
    chosen_move = move

return chosen_move
```

### Task 6

Now it is time to evaluate your two new bots: utility and probability. Run a number of tournaments in the next cell. 
Summarise what you did, and what the results were. 

What I did in the previous code, is also take into consideration the risk of losing a card and make the move based on that. The utility bot seems to improve massively upon the previous, probability bot. Now it wins most of the times against probability, rand and kbbot, though not as well against rdeep. That taken into account, it is safe to say that the more factors are taken into consideration, the higher the performance of the bot.

### Task 7

Again, unless we are much mistaken, the results will be disappointing for both bots. The final task is to change one of the bots and try to improve it. 
In the next cell, describe what changes you attempted to make, add the code that you have changed or added, and report on the tournaments you ran (what did you do, and what were the results). 

Note: do not despair, this is not a simple task and chances are that you will not manage to improve performance much. But still, describe what you have tried. 

I tried to not play the max utility card every time, rather a more optimal card or the maxUtility one. For this I added some code to keep track of a min utility card.
```
minUtility = 9999999
if (move[0] is not None and utility > maxUtility):
    maxUtility = utility
    chosen_move = move
elif (move[0] is not None and utility < minUtility):
    minUtility = utility
```
Now I would check wether a card's utility is between minUtility and maxUtility.
```
if (move[0] is not None and minUtility > utility < maxUtility):
    chosen_move = move
```
The changes I have made have mainly been tested against rdeep bot since that on seems to beat utility bot most and with the largest margin. While testing, I noticed that adding the score instead of multiplying by it seems to reduce the loss margin.
```
if (move[0] is not None):
    rank = move[0] % 5
    points = score[rank]
    utility += points
else:
    utility = 0.0
```
While this strategy is pretty far from winning against rdeep, it managed to reduce the loss margin against rdeep, with the average score at the end of tournaments between rdeep and utility being: __rdeep - average of 12__, __utility - average of 5__.
Compared to previous results: __rdeep - average of 17__, __utility - average of 2__.